In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick # For specifying the axes tick format 
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

plt.style.use('ggplot')

In [2]:
df = pd.read_csv('rfm_merged_df.csv')

In [6]:
df.columns

Index(['date', 'customer_id', 'transaction_id', 'sku_category', 'sku',
       'quantity', 'sales_amount', 'gender', 'age', 'country', 'recency',
       'frequency', 'monetary', 'cluster', 'age_group'],
      dtype='object')

In [9]:
df['date'] = pd.to_datetime(df['date']) 

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131706 entries, 0 to 131705
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            131706 non-null  datetime64[ns]
 1   customer_id     131706 non-null  int64         
 2   transaction_id  131706 non-null  int64         
 3   sku_category    131706 non-null  object        
 4   sku             131706 non-null  object        
 5   quantity        131706 non-null  int64         
 6   sales_amount    131706 non-null  int64         
 7   gender          131706 non-null  int64         
 8   age             131706 non-null  int64         
 9   country         131706 non-null  object        
 10  recency         131706 non-null  int64         
 11  frequency       131706 non-null  int64         
 12  monetary        131706 non-null  int64         
 13  cluster         131706 non-null  int64         
 14  age_group       131706 non-null  obj

In [11]:
df.describe()

,date,customer_id,transaction_id,quantity,sales_amount,gender,age,recency,frequency,monetary,cluster
count,131706,131706.000000,131706.000000,131706.000000,131706.000000,131706.000000,131706.000000,131706.000000,131706.000000,131706.000000,131706.000000
mean,2021-07-04 21:39:26.379663872,12386.450367,32389.604187,1.484222,11.504017,0.444703,35.423823,109.407719,9.027227,279.742123,1.466721
min,2021-01-02 00:00:00,1.000000,1.000000,0.000000,0.000000,0.000000,18.000000,1.000000,1.000000,0.000000,0.000000
25%,2021-04-05 00:00:00,7349.000000,16134.000000,1.000000,4.000000,0.000000,31.000000,16.000000,2.000000,39.000000,0.000000
50%,2021-07-02 00:00:00,13496.000000,32620.000000,1.000000,6.000000,0.000000,36.000000,53.000000,5.000000,116.000000,1.000000
75%,2021-10-07 00:00:00,17306.000000,48548.000000,1.000000,12.000000,1.000000,40.000000,211.000000,11.000000,334.000000,3.000000
max,2021-12-31 00:00:00,22625.000000,64682.000000,400.000000,707.000000,1.000000,62.000000,364.000000,99.000000,3920.000000,3.000000
std,NaN,6086.447552,18709.901238,3.872966,19.350756,0.496935,7.064370,112.797624,11.620495,431.194033,1.172210


In [13]:
user_item_matrix = df.pivot_table(index='customer_id', columns='sku',
                                  values='sales_amount', aggfunc='sum', fill_value=0)


In [14]:
user_item_matrix

sku,00GVC,00OK1,0121I,01IEO,01IQT,01L05,01V7M,01XVY,029WA,03C6L,...,ZWNCK,ZWWB5,ZX2UM,ZY9N9,ZYF2U,ZZ2AO,ZZDW7,ZZM1A,ZZNC5,ZZX6K
customer_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22621,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22622,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22623,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
normalized_user_item_matrix = user_item_matrix.div(user_item_matrix.max(axis=1), axis=0)

In [16]:
normalized_user_item_matrix

sku,00GVC,00OK1,0121I,01IEO,01IQT,01L05,01V7M,01XVY,029WA,03C6L,...,ZWNCK,ZWWB5,ZX2UM,ZY9N9,ZYF2U,ZZ2AO,ZZDW7,ZZM1A,ZZNC5,ZZX6K
customer_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22621,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22622,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
item_user_matrix = user_item_matrix.transpose()

In [18]:
item_user_matrix

customer_id,1,2,3,4,5,6,7,8,9,10,...,22616,22617,22618,22619,22620,22621,22622,22623,22624,22625
sku,,,,,,,,,,,,,,,,,,,,,
00GVC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00OK1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0121I,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01IEO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01IQT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZ2AO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZZDW7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZZM1A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
from sklearn.neighbors import NearestNeighbors

# Initialize the NearestNeighbors model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5, n_jobs=-1)

# Fit the model on the item-user matrix
model_knn.fit(item_user_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1)

In [41]:
item_labels = item_user_matrix.index.tolist()
item_labels = np.array(item_labels)

In [42]:
def make_recommendation(item_id, item_user_matrix, model_knn, item_labels, n_recommendations=5):
    # Find the index of the item_id in the item_labels list
    try:
        item_idx = np.where(item_labels == item_id)[0][0]
    except:
        print(f"Item ID {item_id} not found in item_labels")
        return []
    
    item_features = item_user_matrix.iloc[item_idx, :].values.reshape(1, -1)
    
    # Use the model to find the n_recommendations most similar items
    distances, indices = model_knn.kneighbors(item_features, n_neighbors=n_recommendations+1)
    
    # Convert the indices to item IDs
    recommendations = item_labels[indices.flatten()][1:]  # Exclude the item itself
    
    return recommendations

In [60]:
top_10_skus = df['sku'].value_counts().nlargest(10).index.tolist()
top_10_skus

['UNJKW',
 'COWU2',
 'OV1P9',
 'M6J9W',
 'C6TXL',
 'CZUZX',
 'W1ZMG',
 'TL4WZ',
 'QGK3S',
 'DWJC4']

In [61]:
recommendations = {}

for sku in top_10_skus:    
    recommended_skus = make_recommendation(item_id=sku,
                                                item_user_matrix=item_user_matrix,
                                                model_knn=model_knn,
                                                item_labels=item_labels,
                                                n_recommendations=5)
    recommendations[sku] = recommended_skus

for sku, recommended_skus in recommendations.items():
    print(f"'sku': '{sku}'; 'recommended_sku':{recommended_skus}")


'sku': 'UNJKW'; 'recommended_sku':['KE0PX' 'COWU2' 'K68EA' 'R1MNJ' '6JXQ3']
'sku': 'COWU2'; 'recommended_sku':['B43JA' 'Q7D9D' '1DDOX' 'M6J9W' '2BDVN']
'sku': 'OV1P9'; 'recommended_sku':['G8OED' '2YGR5' '8TZ00' 'IP5IL' 'LPWN0']
'sku': 'M6J9W'; 'recommended_sku':['COWU2' 'N8TYX' 'KZOT8' 'TO211' 'JJLXQ']
'sku': 'C6TXL'; 'recommended_sku':['9Y7M0' 'BEZWI' 'B87MU' '23Y9E' 'JO2C3']
'sku': 'CZUZX'; 'recommended_sku':['5AYE5' '9TR1J' '6B0NN' '8KDM4' '18D11']
'sku': 'W1ZMG'; 'recommended_sku':['I82OJ' 'T2V44' '3103X' 'FLDUC' 'SN3ZJ']
'sku': 'TL4WZ'; 'recommended_sku':['GGYTB' 'CTN07' 'I1PEZ' '1SPIM' 'I8EDF']
'sku': 'QGK3S'; 'recommended_sku':['VQN23' 'XFAEF' '3UH0F' 'PWG2Z' 'EQVEN']
'sku': 'DWJC4'; 'recommended_sku':['3YDVF' '7J2DT' 'J31DY' '7YBOX' 'MM77N']
